In [ ]:
import sys

import gurobipy as gp
import numpy as np
from gurobipy import GRB
from matplotlib import pyplot as plt
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
sys.path.append('../../src')
from ml2gurobi.tree import DecisionTree2Gurobi

In [ ]:
x = np.arange(-1, 1, 0.005)

z = (x)**2

In [ ]:
plt.plot(x, z)

plt.show()

In [ ]:
X = np.concatenate([x.ravel().reshape(-1, 1)], axis=1)
Y = z.ravel()

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)

In [ ]:
# Run our regression
regression = DecisionTreeRegressor(max_depth=10)
regression.fit(X=X_train, y=y_train)

In [ ]:
plt.plot(X, regression.predict(X))

plt.show()

In [ ]:
node = 0
tree = regression.tree_
x = X[200, :]
while tree.children_left[node] > 0:
    if x[tree.feature[node]] <= tree.threshold[node]:
        node = tree.children_left[node]
    else:
        node = tree.children_right[node]

In [ ]:
regression.predict(X[200:201, :])

### Do the optimization model

In [ ]:
npoints = 10
optfeat = [0]

In [ ]:
# Start with classical part of the model
m = gp.Model()

x = m.addMVar((npoints, len(optfeat)), lb=-1, ub=1, name='x')
y = m.addMVar((npoints, 1), lb=-GRB.INFINITY, name='y')

m.setObjective(y.sum(), gp.GRB.MINIMIZE)

In [ ]:
dt2gurobi = DecisionTree2Gurobi(regression, m)
dt2gurobi.predict(x, y)

### Finally optimize it

In [ ]:
m.Params.TimeLimit = 200
m.Params.MIPGap = 0.01

In [ ]:
m.optimize()

In [ ]:
x.X

### Look at the solution

In [ ]:
x.X

In [ ]:
y.X

In [ ]:
regression.predict(x.X)

In [ ]:
tree.value

Copyright © 2020 Gurobi Optimization, LLC